In [ ]:
using Pkg
using Revise

using FileIO
using Logging
using Plots
using Printf
using Statistics
using IntervalSets
using Unitful
using Unitful: ns, ms, µs, s, Hz, kHz, MHz, GHz, THz, m

using DSP
using FFTW

using HTTP
using JSON

using SampleArrays
using Loudspeakers
using PortAudioBindings

pyplot() # the default GR is slow for large graphs

The aim of this notebook is to decide and validate all steps needed for full directivty measurements using measurement turntable.

## TODOs
* `alignsignals`, `finddelay`, ... for `SampleArrays` - mostly done - check what is missing
* make `expsweep.jl` better integrated with `SampleArrays`, i.e. work directly with them; check whole code according to the paper
* SampleArrays - tests, documentation + possibility to name channels?
* Replace Ecasound recording by own solution (ALSA or PulseAudio) - can't solve PortAudio's output underruns :(
* Shouldn't `miccalibrate` and similar methods work with logs (dBs) for better numerical stability? What about different storage options for amplitude/dBs? 

If FFTW crashes, try rebuilding with MKL: https://github.com/JuliaMath/FFTW.jl

In [ ]:
c = 344m/s
samplerate = 44100Hz
calfile = "../data/34Q956_cal_0degree.txt"
stimulus = expsweep(1s; samplerate=samplerate, gain=0.9, optimize=false)
xstimulus = SampleArray(stimulus.sig, samplerate)

In [ ]:
plot(xstimulus)

In [ ]:
# ecasound needs this strange conversion
fstimulus_orig = "../data/stimulus_1s.wav"
fstimulus = "../data/stimulus_1s_fp.wav"
writewav(xstimulus, fstimulus_orig)
run(`sox $fstimulus_orig -e floating-point $fstimulus`)

In [ ]:
function ecasound_ir(fstimulus, fresponse; s=2)
    cmd = `ecasound -t:$s -f:16,2,44100 -a:1 -i $fstimulus -o alsa,pulse -a:2 -i alsa,pulse -o $fresponse`
    run(pipeline(cmd, devnull))
    readwav(fresponse)
end

fresponse = "../data/response_1s.wav"
fresponse2 = "../data/response_1s_2.wav"

In [ ]:
xresponse = ecasound_ir(fstimulus, fresponse; s=2)[0s..1.5s, :]
xresponse2 = ecasound_ir(fstimulus, fresponse2; s=2)[0s..1.5s, :]

In [ ]:
xresponse = readwav(fresponse)
xresponse2 = readwav(fresponse2)
Xresponse = rfft(xresponse)
Xresponse2 = rfft(xresponse2)

# fix the mic measured channel using mic calibration file
Xresponse[:, 2:2] .= miccalibrate(Xresponse[:, 2:2], calfile)
Xresponse2[:, 2:2] .= miccalibrate(Xresponse2[:, 2:2], calfile)
xresponse = irfft(Xresponse)
xresponse2 = irfft(Xresponse2)

In [ ]:
plot(xresponse[:, 1:1])
plot!(50xresponse[:, 2:2])

In [ ]:
# both responses are not aligned due to varying processing delays
plot(xresponse[30000:30100, 1:1], m=:x)
plot!(xresponse2[30000:30100, 1:1], m=:+)

# plot!(100xresponse[:, 2:2])

In [ ]:
# sample difference caused by processing delay
DSP.finddelay(xresponse[:, 2:2], xresponse2[:, 2:2])

In [ ]:
# loopback vs mic - should be same for both recordings for the same distance
@show tdiff = DSP.finddelay(xresponse[:, 2:2], xresponse[:, 1:1]);
@show tdiff2 = DSP.finddelay(xresponse2[:, 2:2], xresponse2[:, 1:1]);

In [ ]:
# distance in ms
uconvert(u"ms", tdiff/samplerate)

In [ ]:
# distance in mm
uconvert(u"mm", c * tdiff/samplerate)

In [ ]:
# find loopback delay caused by the processing
# shift signals, keeping only mic channel
tdiff = DSP.finddelay(xstimulus, xresponse[:, 1:1])
xresponse = DSP.shiftsignal(xresponse, tdiff)[:, 2:2]
tdiff2 = DSP.finddelay(xstimulus, xresponse2[:, 1:1])
xresponse2 = DSP.shiftsignal(xresponse2, tdiff2)[:, 2:2]

In [ ]:
Xstimulus = rfft(xstimulus)
Xresponse = rfft(xresponse)

In [ ]:
h = analyze(stimulus, xresponse)
H = rfft(h)

In [ ]:
# This is a general function which gives IR based on stimulus and the recorded response, some additional filtering should be done
# It is better to use "analyze" below which is 
function ir(xstimulus, xresponse; align=false)
    @assert nchannels(xstimulus) == 1
    @assert nchannels(xresponse) == 1
    if align
        xr, shift = alignsignals(data(xresponse)[:], data(xstimulus)[:])
        xresponse = SampleArray(reshape(xr, :, 1), xresponse.rate)
    end
    
    @assert nframes(xresponse) >= nframes(xstimulus)

    xstimulus = rpad(xstimulus, nframes(xresponse) - nframes(xstimulus))
    Xstimulus = rfft(xstimulus)
    Xresponse = rfft(xresponse)
    H = Xresponse ./ Xstimulus
    h = irfft(H)
    h, H
end

h2, H2 = ir(xstimulus, xresponse2; align=false)

In [ ]:
plot(h[0ms..10ms, :]; label="Farina")
plot!(h2[0ms..10ms, :]; label="general")

In [ ]:
plot(H; label="Farina")
plot!(H2; label="general")

## Remote Table Control

In [ ]:
function make_API_call(url)
    try
        response = HTTP.get(url)
        return JSON.parse(String(response.body))
    catch e
        return "error occurred : $e"
    end
end

table_url = "http://192.168.0.17:5000"
make_API_call("$(table_url)/state")

In [ ]:
make_API_call("$(table_url)/reset")

In [ ]:
make_API_call("$(table_url)/rotate/-355")

In [ ]:
make_API_call("$(table_url)/rotateto/0")

In [ ]:
make_API_call("$(table_url)/speed/0.1")

## Measure Sequence

In [ ]:
function measure(stimulus, fstimulus, fresponse; postduration=1s, c = 344m/s)  
    #TODO fix API so stimulus is not always converted....
    xstimulus = SampleArray(stimulus.sig, stimulus.samplerate)
    duration = getduration(xstimulus)
    xresponse = ecasound_ir(fstimulus, fresponse; s=(duration + postduration))
    measure_info(xresponse)
    xresponse
end

function measure_info(xresponse)
    resp_duration = tdiff = uconvert(u"s", getduration(xresponse))
    tdiff_samples = DSP.finddelay(xresponse[:, 2:2], xresponse[:, 1:1]);
    tdiff = uconvert(u"ms", tdiff_samples/rateHz(xresponse))
    tdiff_distance = uconvert(u"mm", c * tdiff)
    @info "recorded $(nframes(xresponse)) samples ($(resp_duration)); mic lags the loopback by $(tdiff_samples) samples ≈ $(tdiff) ≈ $(tdiff_distance)" 
end

function calibrate(xresponse; calfile="../data/34Q956_cal_0degree.txt")
    # TODO make this more general, e.g., sound card calibration
    Xresponse = rfft(xresponse)
    Xresponse[:, 2:2] .= miccalibrate(Xresponse[:, 2:2], calfile)
    xresponse = irfft(Xresponse)
    xresponse, Xresponse
end

function compute_ir(stimulus, xresponse, fimpulse; noncausal=true)
    # find loopback delay caused by the processing
    # shift signals, keeping only the mic channel
    tdiff = DSP.finddelay(xstimulus, xresponse[:, 1:1])
    xresponse = DSP.shiftsignal(xresponse, tdiff)[:, 2:2]
    
    h = analyze(stimulus, xresponse, noncausal=noncausal)
    writewav(h, fimpulse)
    H = rfft(h)
    h, H
end

In [ ]:
c = 344m/s
samplerate = 44100Hz

# minfreq=1Hz  # woofers
minfreq = 800Hz # tweeter

calfile = "../data/34Q956_cal_0degree.txt"
duration = 5s
# driver = "R_W"
driver = "R_T"
orientation = "hor"
# orientation = "ver"
projectdir = "../data/midtweet/$(driver)/$(orientation)"

mkpath(projectdir)
stimulus = expsweep(duration; samplerate=samplerate, gain=0.9, optimize=false, minfreq=minfreq)
xstimulus = 0.9SampleArray(stimulus.sig, stimulus.samplerate)
# ecasound needs this strange conversion
fstimulus_orig = joinpath(projectdir, "stimulus_1s.wav")
fstimulus = joinpath(projectdir, "stimulus_1s_fp.wav")
writewav(xstimulus, fstimulus_orig)
run(`sox $fstimulus_orig -e floating-point $fstimulus`)

In [ ]:
xresponse = measure(stimulus, fstimulus,
        joinpath(projectdir,  "response_init.wav"); 
        postduration=1s, c = 344m/s);

xresponse, _ = calibrate(xresponse; calfile="../data/34Q956_cal_0degree.txt")

h, H = compute_ir(stimulus, xresponse, joinpath(projectdir, "ir_init.wav"); noncausal=true)

In [ ]:
xresponse = readwav(joinpath(projectdir,  "response_init.wav"))
measure_info(xresponse)

* **Midtweet R_W horizontal:** recorded 258048 samples (5.851428571428571 s); mic lags the loopback by 131 samples ≈ 2.9705215419501134 ms ≈ 1021.859410430839 mm
* **Midtweet R_T horizontal:** recorded 258048 samples (5.851428571428571 s); mic lags the loopback by 129 samples ≈ 2.925170068027211 ms ≈ 1006.2585034013606 mm
* **Midtweet R_W vertical:** recorded 258048 samples (5.851428571428571 s); mic lags the loopback by 131 samples ≈ 2.9705215419501134 ms ≈ 1021.859410430839 mm
* not moving mic
* **Midtweet R_T vertical:** recorded 258048 samples (5.851428571428571 s); mic lags the loopback by 130 samples ≈ 2.9478458049886624 ms ≈ 1014.0589569160999 mm

In [ ]:
plot(h)

In [ ]:
plot(h[duration-0ms..duration+10ms, :])

In [ ]:
plot(H)

In [ ]:
make_API_call("$(table_url)/speed/0.1")

In [ ]:
function measure_all(stimulus, fstimulus, rawdir; 
        response_template="response_{deg}.wav",
        degs=0:5:355, rate=44100Hz, duration=5s, speed=0.1)
    make_API_call("$(table_url)/speed/$(speed)")
    mkpath(rawdir)
    for deg in degs
        @info "measurement for $(deg)°"
        make_API_call("$(table_url)/rotateto/$(deg)")
        sleep(2) # wait for damping vibrations
        fresponse = joinpath(rawdir, replace(response_template, "{deg}" => @sprintf("%03d", deg)))
        isfile(fresponse) && error("response file already exists: $(fresponse)!")
        
        measure(stimulus, fstimulus, fresponse; postduration=1s, c = 344m/s) 
    end
end

In [ ]:
# sleep(10) # run away :)

measure_all(stimulus, fstimulus,
    joinpath(projectdir, "raw"); 
    degs=0:5:355, rate=44100Hz, duration=duration)

In [ ]:
function postprocess(rawdir, stimulus, irdir; 
    response_template=r"response_(\d\d\d).wav", 
    ir_template="ir_{deg}.wav",
    calfile="../data/34Q956_cal_0degree.txt")
    
    mkpath(irdir)
    
    for f in readdir(rawdir)
        fpath = joinpath(rawdir, f)
        if isfile(fpath)
            m = match(response_template, f)
            if !isnothing(m)
                deg = parse(Int, m.captures[1])
                if deg < 0
                    deg = 360 - deg
                end
                @info "processing: $fpath"
                xresponse = readwav(fpath)
                if !isnothing(calfile)
                    xresponse, _ = calibrate(xresponse; calfile=calfile)
                end
                fimpulse = joinpath(irdir, replace(ir_template, "{deg}" => @sprintf("%03d", deg)))
                isfile(fimpulse) && error("impulse file already exists: $(fimpulse)!")

                h, H = compute_ir(stimulus, xresponse, fimpulse; noncausal=true)
            end
        end
    end
end
    
postprocess(
        joinpath(projectdir, "raw"),
        stimulus,
        joinpath(projectdir, "ir"), 
        calfile="../data/34Q956_cal_0degree.txt")

## Analysis

In [ ]:
projectdir

In [ ]:
irs = [(deg, readwav(joinpath(projectdir, replace("ir/ir_{deg}.wav", "{deg}" => @sprintf("%03d", deg)))))
    for deg in 0:10:355];

In [ ]:
peakidx = irfirstbig(irs[1][2], 0.3)

In [ ]:
plot(
    [plot(ir[peakidx-200:peakidx+400,:], label="$(deg)°") for (deg, ir) in irs]..., 
    layout=(length(irs), 1), size=(1200, 200length(irs))
)

Find the quasi anechoic setup based on the 0° IR.

In [ ]:
x0 = readwav(joinpath(projectdir, "ir/ir_000.wav"))

In [ ]:
peakidx = irfirstbig(x0, 0.3)

In [ ]:
x0b = x0[peakidx-100:peakidx+100,:]

In [ ]:
plot(x0b)
vline!([domain(x0b)[irfirstbig(x0b, 0.3)]])

In [ ]:
function load_farfield(fimpulse; peakf=x->peakidx, post=4.0ms)
    x = readwav(fimpulse)
    x = quasi_anechoic(x, peakf; post=post, pre=125ms, cut=true)
    X = rfft(x)
    return (x=x, X=X)
end

x, X = load_farfield(joinpath(projectdir, "ir/ir_000.wav"))

In [ ]:
plot(x)
# plot(x[5s-5ms..5s+5ms,:])

In [ ]:
plot(X; xlims=(1000,Inf), ylims=(-50,Inf))

In [ ]:
ds

In [ ]:
directivitymap(ds; levels=30)

In [ ]:
directivityplot(ds, [10000Hz]; minclip=-80)

In [ ]:
ds[:, 0..45].angles

In [ ]:
plot(ds[0.0, 45.0].spectrum; xlims=(1000, Inf), ylims=(-50,Inf))

In [ ]:
ds[0.0, 45.0]